### Create a config table and insert some data into it
This holds the information of the source delta table and the SQL target table

### Create and insert data into the server detail table

In [0]:
%sql
CREATE OR REPLACE table sqlsrvr_dtl_config
(server_id BIGINT, hostname STRING, key_scope STRING, cred_type STRING, credentials STRUCT<userid STRING, password STRING>);

ALTER TABLE sqlsrvr_dtl_config ADD CONSTRAINT cred_type_chk CHECK(cred_type IN ('useridpassword','serviceprincipal'));

INSERT INTO sqlsrvr_dtl_config
VALUES
  (1, "synapsews-jrisng.sql.azuresynapse.net", "jriscopekv", "useridpassword", PARSE_JSON('{"userid":"synapseuser2","password":"synapsepassword2"}'));

### Create and insert data into the database detail table

In [0]:
%sql
CREATE OR REPLAce table sqldb_dtl_config
(db_id BIGINT, db_name STRING, server_id BIGINT);

INSERT INTO sqldb_dtl_config
VALUES
  (1, "testpool", 1);

### Create and insert data into the sql load config detail table

In [0]:
%sql
CREATE OR REPLACE TABLE sql_load_config
  (id BIGINT, sql_type STRING, sql_group STRING, src_table STRING, trgt_table STRING, db_id BIGINT);

INSERT INTO sql_load_config
VALUES 
  (1, "sqlpool", "TESTCASEA", "table_25mb", "table_25mb1a", 1), 
  (2, "sqlpool", "TESTCASEA", "table_25mb", "table_25mb2a", 1),
  (3, "sqlpool", "TESTCASEA", "table_25mb", "table_25mb3a", 1),
  (4, "sqlpool", "TESTCASEA", "table_25mb", "table_25mb4a", 1),
  (5, "sqlpool", "TESTCASEB", "table1", "table1", 1),
  (6, "sqlpool", "TESTCASEB", "table2", "table2", 1);

### Create view which will be used by the sql load framework

In [0]:
%sql
CREATE OR REPLACE VIEW v_sql_load_config AS
SELECT A.*, B.db_name, B.server_id,C.hostname, C.key_scope, C.cred_type, C.credentials FROM sql_load_config A 
LEFT JOIN sqldb_dtl_config B ON B.db_id = A.db_id
LEFT JOIN sqlsrvr_dtl_config C ON C.server_id = B.server_id;

SELECT * FROM v_sql_load_config